<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/nvidia_triton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


Nvidia Triton是一个用于部署和推理机器学习模型的开源软件。它提供了一个统一的接口，可以用于部署训练好的模型，并支持多种深度学习框架，包括TensorFlow、PyTorch和ONNX等。Triton还提供了高性能推理的能力，可以在GPU和CPU上进行推理，并支持多个模型同时部署。Nvidia Triton还包括了用于监控和管理模型部署的工具，使得模型的部署和管理变得更加简单和高效。


[NVIDIA Triton Inference Server](https://github.com/triton-inference-server/server) 提供了针对CPU和GPU进行优化的云端和边缘推理解决方案。此连接器允许 llama_index 与使用 Triton 部署的 TRT-LLM 模型进行远程交互。


## 启动Triton推理服务器


这个连接器需要一个运行中的Triton推理服务器，配备一个TensorRT-LLM模型。
在这个示例中，我们将使用[Triton命令行界面（Triton CLI）](https://github.com/triton-inference-server/triton_cli)在Triton上部署一个GPT2模型。

当在主机上使用Triton和相关工具（而不是在Triton容器映像之外）时，可能需要一些额外的依赖项来支持各种工作流程。大多数系统依赖问题可以通过在最新对应的`tritonserver`容器映像内安装并运行CLI来解决，该容器应该安装了所有必要的系统依赖项。

对于TRT-LLM，你可以使用`nvcr.io/nvidia/tritonserver:{YY.MM}-trtllm-python-py3`映像，其中`YY.MM`对应于`tritonserver`的版本，例如在这个示例中我们使用了24.02版本的容器。要获取可用版本的列表，请参考[Triton推理服务器NGC](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver)。

要启动容器，请在Linux终端中运行：

```
docker run -ti --gpus all --network=host --shm-size=1g --ulimit memlock=-1 nvcr.io/nvidia/tritonserver:24.02-trtllm-python-py3
```
接下来，我们需要使用以下命令安装依赖项：
```
pip install \
  "psutil" \
  "pynvml>=11.5.0" \
  "torch==2.1.2" \
  "tensorrt_llm==0.8.0" --extra-index-url https://pypi.nvidia.com/
```
最后，运行以下命令安装Triton CLI。


```
pip install git+https://github.com/triton-inference-server/triton_cli.git
```


生成GPT2模型的模型库并启动Triton服务器实例：
```
triton remove -m all
triton import -m gpt2 --backend tensorrtllm
triton start &
```
请注意，默认情况下，Triton开始监听`localhost:8000` HTTP端口和`localhost:8001` GRPC端口。后者将在本示例中使用。
如需任何其他操作指南和问题，请联系[Triton命令行界面（Triton CLI）](https://github.com/triton-inference-server/triton_cli)的问题页面。


## 安装tritonclient
由于我们将与Triton推理服务器进行交互，因此我们需要[安装](https://github.com/triton-inference-server/client?tab=readme-ov-file#download-using-python-package-installer-pip) `tritonclient` 包。


```
pip install tritonclient[all]
```


接下来，我们将安装llama索引连接器。
```
pip install llama-index-llms-nvidia-triton
```


## 基本用法


#### 使用提示调用`complete`


```python
from llama_index.llms.nvidia_triton import NvidiaTriton

# 必须运行一个Triton服务器实例。使用您所需的Triton服务器实例的正确URL。
triton_url = "localhost:8001"
model_name = "gpt2"
resp = NvidiaTriton(server_url=triton_url, model_name=model_name, tokens=32).complete("北美洲最高的山是 ")
print(resp)
```


你应该期待以下的回复
```
吉萨大金字塔，高约1,000英尺。吉萨大金字塔是北美最高的山。
```


#### 使用 prompt 调用 `stream_complete`


```python
resp = NvidiaTriton(server_url=triton_url, model_name=model_name, tokens=32).stream_complete("北美洲最高的山是 ")
for delta in resp:
    print(delta.delta, end=" ")
```


您应该期望以下作为流的响应
```
吉萨大金字塔，高约1,000英尺。吉萨大金字塔是北美最高的山。
```


## 更多示例
有关Triton推理服务器的更多信息，请参考[快速入门](https://github.com/triton-inference-server/server/blob/main/docs/getting_started/quickstart.md#quickstart)指南，[NVIDIA开发者Triton页面](https://developer.nvidia.com/triton-inference-server)，以及[GitHub问题](https://github.com/triton-inference-server/server/issues)频道。
